In [1]:
import pandas as pd
from tensorflow.keras.models import load_model
import pickle
import numpy as np

In [3]:
## load the trained model, scaler pickle,onehot
model = load_model('model.h5')

#load the encoder and scaler
with open('label.pkl','rb') as file:
    label = pickle.load(file)

with open('onehot.pkl','rb') as file:
    onehot = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)


In [4]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [7]:
geo_encoded = onehot.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns = onehot.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\pooja\OneDrive\Documents\Gen Ai course\Ann Classification\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [8]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [11]:
input_df['Gender'] = label.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [13]:
#concatenation

input_df = pd.concat([input_df.drop("Geography",axis = 1),geo_encoded_df],axis =1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [14]:
#Scaling the data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [15]:
#Prediction
prediction  =model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step


array([[0.02692988]], dtype=float32)

In [21]:
predprobs = prediction[0][0]


In [22]:
predprobs

0.026929878

In [25]:
if predprobs >0.5:
    print("The customer is likely to Churn")
else:
    print("The customer is not likely churn")

The customer is not likely churn
